In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [111]:
df = pd.read_csv("Bank_Churn.csv")
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [112]:
df['AgeGroup'] = pd.cut(df['Age'], bins=[18, 25, 35, 45, 60,120], labels=['18-25', '26-35', '36-45', '46-60', '60+'],right=False)
df['CreditScoreCat'] = pd.cut(df['CreditScore'], 
                                   bins=[300, 580, 669, 739, 799, 850], 
                                   labels=['Poor', 'Fair', 'Good', 'Very Good', 'Exceptional'])


In [113]:
new_df = df.drop(columns = ['CustomerId','Surname','CreditScore','Age'])
new_df 

,Geography,Gender,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,AgeGroup,CreditScoreCat
0,France,Female,2,0.00,1,1,1,101348.88,1,36-45,Fair
1,Spain,Female,1,83807.86,1,0,1,112542.58,0,36-45,Fair
2,France,Female,8,159660.80,3,1,0,113931.57,1,36-45,Poor
3,France,Female,1,0.00,2,0,0,93826.63,0,36-45,Good
4,Spain,Female,2,125510.82,1,1,1,79084.10,0,36-45,Exceptional
...,...,...,...,...,...,...,...,...,...,...,...
9995,France,Male,5,0.00,2,1,0,96270.64,0,36-45,Very Good
9996,France,Male,10,57369.61,1,1,1,101699.77,0,36-45,Poor
9997,France,Female,7,0.00,1,0,1,42085.58,1,36-45,Good
9998,Germany,Male,3,75075.31,2,1,0,92888.52,1,36-45,Very Good


In [100]:
new_df.shape

(10000, 11)

In [114]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(new_df.drop(columns=['Exited']) ,new_df['Exited'],test_size = 0.3, random_state = 0)
X_train

,Geography,Gender,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,AgeGroup,CreditScoreCat
7681,France,Male,2,146193.60,2,1,1,55796.83,26-35,Fair
9031,France,Male,7,0.00,2,1,0,19823.02,36-45,Poor
3691,France,Female,5,160979.68,1,0,1,13848.58,60+,Fair
202,Spain,Male,5,0.00,1,0,1,146145.93,46-60,Poor
5625,France,Female,7,143262.04,1,1,1,129562.74,60+,Poor
...,...,...,...,...,...,...,...,...,...,...
9225,Germany,Female,4,120074.97,2,1,1,162961.79,26-35,Fair
4859,Spain,Female,4,114440.24,1,1,1,107753.07,18-25,Very Good
3264,France,Male,5,161274.05,2,1,0,181429.87,36-45,Good
9845,Spain,Female,9,0.00,2,1,1,148750.16,36-45,Fair


In [116]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [103]:

tnf1 = ColumnTransformer(transformers=[
    ('scaling', StandardScaler(), [3, 7]),  
    ('one_hot', OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore'), [0, 1]),  
    ('credit_encode', OrdinalEncoder(categories=[['Poor', 'Fair', 'Good', 'Very Good', 'Exceptional']]), [9]), 
    ('age_encode', OrdinalEncoder(categories=[['18-25', '26-35', '36-45', '46-60', '60+']]), [8])  
], remainder='passthrough')  


In [104]:
tnf2 = LogisticRegression(random_state=42)

In [105]:
pipe = Pipeline([
    ('transformer', tnf1),
    ('model', tnf2)
])

In [106]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaling', StandardScaler(),
                                                  [3, 7]),
                                                 ('one_hot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [0, 1]),
                                                 ('credit_encode',
                                                  OrdinalEncoder(categories=[['Poor',
                                                                              'Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Exceptional']]),
                                                  [9]),
                                                 ('age_encode',
                                                  OrdinalEncoder(categories=[['18-25',
                                                                              '26-35',
                                                                              '36-45',
                                                                              '46-60',
                                                                              '60+']]),
                                                  [8])])),
                ('model', LogisticRegression(random_state=42))])

In [110]:
y_pred = pipe.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [108]:
from sklearn import set_config
set_config(display = 'diagram')

In [109]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.816

In [117]:

tnf1 = ColumnTransformer(transformers=[
    ('scaling', StandardScaler(), [3, 7]),  
    ('one_hot', OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore'), [0, 1]),  
    ('credit_encode', OrdinalEncoder(categories=[['Poor', 'Fair', 'Good', 'Very Good', 'Exceptional']]), [9]), 
    ('age_encode', OrdinalEncoder(categories=[['18-25', '26-35', '36-45', '46-60', '60+']]), [8])  
], remainder='passthrough')  


In [118]:
tnf2 = DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=42)

In [119]:
pipe = Pipeline([
    ('transformer', tnf1),
    ('model', tnf2)
])

In [120]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaling', StandardScaler(),
                                                  [3, 7]),
                                                 ('one_hot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [0, 1]),
                                                 ('credit_encode',
                                                  OrdinalEncoder(categories=[['Poor',
                                                                              'Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Exceptional']]),
                                                  [9]),
                                                 ('age_encode',
                                                  OrdinalEncoder(categories=[['18-25',
                                                                              '26-35',
                                                                              '36-45',
                                                                              '46-60',
                                                                              '60+']]),
                                                  [8])])),
                ('model',
                 DecisionTreeClassifier(max_depth=4, random_state=42))])

In [121]:
y_pred = pipe.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [122]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.8586666666666667

Here, I tried to make the pipeline. I followed the code just like I learned, It's working, but I am still unsure if this is correct. Also, I found out what was wrong with NaN values in the Age Group Column. Actually, Pd.cut was introducing those values because I didn't handle the categories well. I completely forgot about the interval inclusive and exclusive logic. I have corrected the code and have applied the pipeline step by step. Since the decision tree gives me an 85% accuracy score, I will stick to it. I know there are more steps involved like cross-validation and optimization of the model but honestly I have not started with it yet. So I won't apply it here just for the sake of it. I want to have an intuition why am I doing what so I will go back to a different dataset and try to conduct EDA and model fitting on it. 